In [1]:
import os, re, time, json
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
import tensorflow as tf
import google
from matplotlib import pyplot as plt
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.3.0


In [2]:
import tensorflow as tf
import tensorflow.keras as keras

from tf_records_generator import get_dataset
from efficientNet_B0 import EffNet0


In [3]:
CONFIG = dict(
    replicas=1,
    input_shape=(256, 256, 3),
    image_resolution=256,
    batch_size=8,
    epochs=5,
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
    output_bias=np.log([584/32542]),
    weight_for_0 = (1 / 32542)*(32542+584)/2.0,
    weight_for_1 = (1 / 584)*(32542+584)/2.0
)

In [4]:
IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
  from google.colab import auth
  # Authenticates the Colab machine and also the TPU using your
  # credentials so that they can access your private GCS buckets.
  auth.authenticate_user()
  data_dir = 'gs://dataset_files/'
else:
  data_dir = 'dataset/'

In [5]:
# Detect hardware
try:
  tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu_resolver = None
  gpus = tf.config.experimental.list_logical_devices("GPU")

# Select appropriate distribution strategy
if tpu_resolver:
  tf.config.experimental_connect_to_cluster(tpu_resolver)
  tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
  strategy = tf.distribute.TPUStrategy(tpu_resolver)
  print('Running on TPU ', tpu_resolver.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
  
print("Number of accelerators: ", strategy.num_replicas_in_sync)
CONFIG["replicas"] = strategy.num_replicas_in_sync


INFO:tensorflow:Initializing the TPU system: grpc://10.6.16.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.6.16.82:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  ['10.6.16.82:8470']
Number of accelerators:  8


In [6]:
path = data_dir + str(CONFIG["image_resolution"])
files_train = np.sort(np.array(tf.io.gfile.glob(path + '/train*.tfrec')))
files_test = np.sort(np.array(tf.io.gfile.glob(path + '/test*.tfrec')))


ds = get_dataset(files_train[:-3], CONFIG)
val_ds = get_dataset(files_train[-3], CONFIG)
test_ds = get_dataset(files_test, CONFIG)

Tensor("ParseSingleExample/ParseExample/ParseExampleV2:3", shape=(), dtype=string)
Tensor("ParseSingleExample/ParseExample/ParseExampleV2:3", shape=(), dtype=string)
Tensor("ParseSingleExample/ParseExample/ParseExampleV2:3", shape=(), dtype=string)


In [7]:
with strategy.scope():
  model = EffNet0(config=CONFIG, trainable_base=True)
  model.compile(optimizer=CONFIG["optimizer"], loss=CONFIG["loss"], 
                metrics=[
                              keras.metrics.TruePositives(name='tp'),
                              keras.metrics.FalsePositives(name='fp'),
                              keras.metrics.TrueNegatives(name='tn'),
                              keras.metrics.FalseNegatives(name='fn'), 
                              keras.metrics.BinaryAccuracy(name='accuracy'),
                              keras.metrics.Precision(name='precision'),
                              keras.metrics.Recall(name='recall'),
                              keras.metrics.AUC(name='auc'),
                        ]
)


16711680/16705208 [==============================] - 0s 0us/step


In [8]:
history = model.fit(ds,class_weight={0:CONFIG["weight_for_0"],1:CONFIG["weight_for_1"]}, validation_data=val_ds,validation_batch_size=CONFIG["batch_size"]*CONFIG["replicas"],
                    batch_size=CONFIG["batch_size"]*CONFIG["replicas"], epochs=50)

Epoch 1/50
408/408 [==============================] - 59s 144ms/step - loss: 7.5986 - tp: 167.0000 - fp: 7178.0000 - tn: 18470.0000 - fn: 297.0000 - accuracy: 0.7137 - precision: 0.0227 - recall: 0.3599 - auc: 0.5753 - val_loss: 0.4072 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 2137.0000 - val_fn: 39.0000 - val_accuracy: 0.9821 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.6031
Epoch 2/50
408/408 [==============================] - 53s 130ms/step - loss: 1.4696 - tp: 221.0000 - fp: 8739.0000 - tn: 16909.0000 - fn: 243.0000 - accuracy: 0.6560 - precision: 0.0247 - recall: 0.4763 - auc: 0.5893 - val_loss: 0.3422 - val_tp: 2.0000 - val_fp: 19.0000 - val_tn: 2118.0000 - val_fn: 37.0000 - val_accuracy: 0.9743 - val_precision: 0.0952 - val_recall: 0.0513 - val_auc: 0.6895
Epoch 3/50
408/408 [==============================] - 55s 134ms/step - loss: 0.7891 - tp: 231.0000 - fp: 8546.0000 - tn: 17102.0000 - fn: 233.0000 - accuracy: 0.6638 - precision: 0.0263 - recall:

KeyboardInterrupt: ignored